# Install

In [ ]:
#build
!git clone https://github.com/facebookresearch/fastText.git
!cd fasttext
!pip install fastText

In [ ]:
import fasttext

In [ ]:
#help : 라이브러리의 상위 파일을 보여준다
help(fasttext.FastText)

# Word Representation
단어를 벡터로 변환하면 word analogies나 word semantic 등등 언어에 숨겨져 있는 정보를 포착할 수 있다. <br>
Fasttext를 사용하여 단어 벡터를 만들어보자.

# Getting the data
* corpus - 1billion bytes of English Wikipedia
* 전처리 - wikifil.pl 사용해서 HTML/XML 데이터 처리 

In [ ]:
#data 폴더 생성
!mkdir data

In [ ]:
#데이터 다운
!wget -c http://mattmahoney.net/dc/enwik9.zip -P data

In [ ]:
#압축 해제
!unzip data/enwik9.zip -d data

In [ ]:
#전처리
!perl fastText/wikifil.pl data/enwik9 > data/fil9

In [ ]:
!head -c 80 data/fil9

# Training word vectors

In [ ]:
import fasttext

In [ ]:
#Learning word vectors(시간 오래 걸림..)
model = fasttext.train_unsupervised('data/fil9')

In [ ]:
model.words

In [ ]:
#Get the word vector
model.get_word_vector("the")

In [ ]:
#Save this model
model.save_model("result/fil9.bin")

In [ ]:
#Reload
model = fasttext.load_model("result/fil9.bin")

# Skipgram VS CBOW
fasttext는 skipgram과 cbow(continuous-bag-of-words) 모델을 제공한다.
* skipgram : nearby word로 target word를 예측.
* cbow : context word로 target word를 예측.
<img src="https://t1.daumcdn.net/cfile/tistory/99C6A23A5C9043C80F">

In [ ]:
#cbow model
cbow_model = fasttext.train_unsupervised('data/fil9','cbow')

# Parameters for word vectors
data parameters
* dim : dimension, 벡터 크기 조절. default = 100 (주로, 100 - 300)
* minn : minimum size for the subwords. default = 3
* maxn : maximal size for the subwords. defualt = 6

train parameters
* epoch : 데이터 반복 횟수 제어. default = 5
* lr : learning rate. default = 0.05 (0.01 - 1 권장)
<br>
* thread : fastText는 멀티스레드를 사용. default = 12

In [ ]:
#data parameter
model = fasttext.train_unsupervised('data/fil9', minn=2, maxn=5, dim=300)

In [ ]:
#train parameter
model = fasttext.train_unsupervised('data/fil9', epoch=1, lr=0.5)

In [ ]:
model = fasttext.train_unsupervised('data/fil9', thread=4)

In [ ]:
model2 = fasttext.train_unsupervised('data/fil9', minn=2, maxn=5, dim=300, epoch=1, thread=4)

# Printing word vectors
데이터에 없는 단어도 벡터 출력 가능

In [ ]:
for x in ['asparagus', 'pidgey', 'yellow']:
  print(model.get_word_vector(x))

In [ ]:
#oov
model.get_word_vector("enviroment")

# Nearest neighbor queries
- word vector의 퀄리티를 평가
- 벡터의 의미 정보 유형을 직관적으로 보여줌.

In [19]:
model.get_nearest_neighbors('asparagus')

[(0.8229429125785828, 'horseradish'),
 (0.8174294829368591, 'cabbage'),
 (0.8091084361076355, 'cauliflower'),
 (0.8013622760772705, 'cabbages'),
 (0.792203426361084, 'beetroot'),
 (0.7917746305465698, 'asparagales'),
 (0.7904706597328186, 'spinach'),
 (0.7856069803237915, 'cauliflowers'),
 (0.781997561454773, 'arrowroot'),
 (0.7778395414352417, 'oleracea')]

In [20]:
model.get_nearest_neighbors('pidgey')

[(0.9040499329566956, 'pidgeot'),
 (0.8955814242362976, 'pidge'),
 (0.8770537972450256, 'pidgeotto'),
 (0.7627281546592712, 'pidgeon'),
 (0.7459321618080139, 'pikachu'),
 (0.7438387274742126, 'squirtle'),
 (0.7122770547866821, 'pokemon'),
 (0.7104504704475403, 'johto'),
 (0.6992117166519165, 'beedrill'),
 (0.6975911259651184, 'butterfree')]

In [21]:
#misspelled word
model.get_nearest_neighbors('enviroment')

[(0.8960695266723633, 'enviromental'),
 (0.8583640456199646, 'environ'),
 (0.8029642701148987, 'enviro'),
 (0.7756938338279724, 'environnement'),
 (0.7536866664886475, 'environs'),
 (0.7470517754554749, 'enviromission'),
 (0.6951451301574707, 'environment'),
 (0.6946241855621338, 'ecotone'),
 (0.6925439238548279, 'sustainability'),
 (0.6888049244880676, 'ecotourism')]

# Measure of similarity
단어 사이의 유사성을 계산해 최근접 이웃을 찾을 수 있다. <br>
모든 단어들을 계산해서 가장 유사한 단어 10개를 나타낸다.
해당 단어가 있다면 상단에 표시되고 유사도는 1이다.

# Word analogies
ex) France, Berlin, Germany 세 단어의 관계를 유추할 수 있다.

In [22]:
model.get_analogies('berlin','germany','france')

[(0.8967704176902771, 'paris'),
 (0.7749188542366028, 'maubourg'),
 (0.757949709892273, 'louveciennes'),
 (0.7532722353935242, 'faubourg'),
 (0.7503136396408081, 'monceaux'),
 (0.7501069903373718, 'valenciennes'),
 (0.7470589280128479, 'volontaires'),
 (0.7469262480735779, 'tourelles'),
 (0.7445774674415588, 'rouvres'),
 (0.7430331110954285, 'beaubourg')]

In [23]:
model.get_analogies('psx','sony','nintendo')

[(0.7564773559570312, 'gamecube'),
 (0.7556949257850647, 'gba'),
 (0.7316620349884033, 'zaxxon'),
 (0.7265994548797607, 'arcade'),
 (0.7215122580528259, 'gamepark'),
 (0.721269965171814, 'gameboy'),
 (0.719158411026001, 'dreamcast'),
 (0.7140929698944092, 'super'),
 (0.7116702198982239, 'sega'),
 (0.7110921144485474, 'snes')]

# Importance of character n-grams
서브단어 정보를 사용하면 모르는 단어의 벡터도 빌드할 수 있다.<br>
ex) gearshift는 위키피티아에 없지만 비슷한 단어를 찾을 수 있다.

In [24]:
model.get_nearest_neighbors('gearshift')

[(0.7953199744224548, 'gears'),
 (0.7944257259368896, 'gearing'),
 (0.7617721557617188, 'flywheels'),
 (0.7587226033210754, 'flywheel'),
 (0.7533383369445801, 'driveshafts'),
 (0.7500735521316528, 'tailwheel'),
 (0.748734712600708, 'daisywheel'),
 (0.742712140083313, 'gearboxes'),
 (0.736818253993988, 'runabouts'),
 (0.7362300157546997, 'driveshaft')]

In [ ]:
#서브단어 없이 학습하는 모델
model_without_subwords = fasttext.train_unsupervised('data/fil9', maxn=0)

In [ ]:
#서브단어가 없는 모델에 스펠링 오류 단어를 넣어 최근접 이웃 출력
model_without_subwords.get_nearest_neighbors('accomodation')

In [25]:
#서브단어 활용 모델
model.get_nearest_neighbors('accomodation')

[(0.9668127298355103, 'accomodations'),
 (0.9419698715209961, 'accommodation'),
 (0.9248396754264832, 'accommodations'),
 (0.8561415672302246, 'accommodative'),
 (0.795039713382721, 'accommodating'),
 (0.7890238761901855, 'amenities'),
 (0.7589897513389587, 'accomodated'),
 (0.754024088382721, 'accomodate'),
 (0.7530205845832825, 'lodging'),
 (0.7515777945518494, 'catering')]